In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM,Dense,Bidirectional

Using TensorFlow backend.


In [2]:
#Sample Univariate sequence
seq = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Supervised Algorightms require the data in the form of input(X) and output(y).
We can write a function to transform a given sequence into input and output 

In [3]:
#split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = [],[]
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return np.array(X),np.array(y)

In [4]:
n_steps=3
X, y = split_sequence(seq,n_steps)

In [5]:
for i in range(len(X)):
  print(X[i],y[i])

[1 2 3] 4
[2 3 4] 5
[3 4 5] 6
[4 5 6] 7
[5 6 7] 8
[6 7 8] 9
[7 8 9] 10


**1. LSTM**

The input to LSTM layer must be three-dimensional

*   Samples
*   Timesteps
*   Fetures

When defining the input layer of your LSTM network, the network assumes you have one or more samples and requires that you specify the number of time steps and the number of features

In [6]:
#Reshape the input data into 3D shape as required by LSTM
n_features = 1
X= X.reshape(X.shape[0],X.shape[1],n_features)

In [7]:
#Define LSTM Model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps,n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [8]:
#Fit Model
model.fit(X,y,epochs=200,verbose=0)

In [9]:
#Prediction
x_test = np.array([[7,8,9]])
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],n_features)
y_hat = model.predict(x_test)
print(y_hat)

[[10.917564]]


**Stacked LSTM model**

LSTMs produce 2D output as an interpretation from end of the sequence. We can change this by LSTM output a value for each timestep in the input data by setting Return Sequence =True

In [10]:
#create LSTM Stacked model
stack_model = Sequential()
stack_model.add(LSTM(50,activation='relu',return_sequences=True, input_shape=(n_steps,n_features)))
stack_model.add(LSTM(50,activation='relu'))
stack_model.add(Dense(1))
stack_model.compile(optimizer='adam',loss='mse')

In [11]:
#Fit model
stack_model.fit(X,y,epochs=200, verbose=0)

In [12]:
y_hat_stack = stack_model.predict(x_test)
print(y_hat_stack)

[[9.788965]]


**Bidirectional LSTM**

In some sequence prediction problems, it is benificial to allow the LSTM model to learn the input sequence both forward and backwards and concatenate both interpretations.


In [13]:
#Define Bidirectional LSTM Model
bidi_model = Sequential()
bidi_model.add(Bidirectional(LSTM(50, activation='relu'),input_shape=(n_steps,n_features)))
bidi_model.add(Dense(1))
bidi_model.compile(optimizer='adam',loss='mse')

In [14]:
bidi_model.fit(X,y,epochs=200,verbose=0)

In [15]:
bidi_model.predict(x_test)

array([[9.742871]], dtype=float32)